# Configuración e Importación de Librerías

En esta sección inicializamos Spark e importamos las librerías necesarias que se utilizarán a lo largo del proyecto.

## Inicialización de Spark
Comenzamos importando e inicializando `findspark` para asegurarnos de que PySpark esté accesible e integrado dentro del notebook. Esto es especialmente útil cuando trabajamos con Spark fuera de su entorno predeterminado (por ejemplo, Databricks).


In [0]:
import findspark
findspark.init()

# PySpark y Manipulación de Datos
El núcleo del proyecto involucra el uso de PySpark para el procesamiento de datos y la ejecución de tareas de machine learning. Importamos pyspark y pandas, lo que nos permitirá manejar dataframes e interactuar con el motor de cómputo distribuido de Spark. El objeto SparkSession es el punto de entrada para interactuar con Spark.

In [0]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

## Librerías de Machine Learning
Importamos varias librerías de machine learning de PySpark que se utilizarán para crear modelos, como LogisticRegression, DecisionTreeClassifier y RandomForestClassifier. Además, importamos Pipeline de PySpark para estructurar nuestro flujo de trabajo de machine learning y MulticlassClassificationEvaluator para evaluar el rendimiento del modelo.

Explicación de los Componentes Importados:
- VectorAssembler: Combina múltiples columnas en un solo vector de características, lo cual es esencial para los modelos de Spark ML.
- PCA (Análisis de Componentes Principales): Utilizado para la reducción de dimensionalidad, mejorando el rendimiento y la eficiencia del modelo.
- LogisticRegression, DecisionTreeClassifier, RandomForestClassifier: Algoritmos de clasificación que se emplearán para crear nuestros modelos de clasificación de vinos.
- Pipeline: Una abstracción de alto nivel en PySpark ML que permite encadenar varios pasos (por ejemplo, transformaciones de datos y modelado) en un flujo de trabajo unificado.
- MulticlassClassificationEvaluator, BinaryClassificationEvaluator: Evalúan el rendimiento de los modelos de clasificación. Aunque las métricas de evaluación binarias son más relevantes para problemas binarios, se adaptarán para problemas de clasificación multiclase.

In [0]:
from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

### Perfilado de Datos
Utilizamos ydata_profiling para realizar un análisis y exploración inicial de los datos. Esta librería genera reportes completos sobre los datos, incluyendo correlaciones, valores faltantes y distribuciones.

In [0]:
from ydata_profiling import ProfileReport

### Validación Cruzada y Ajuste de Hiperparámetros
Para optimizar nuestro modelo, usamos CrossValidator y ParamGridBuilder para realizar validación cruzada y ajustar los hiperparámetros. Esto asegura que nuestro modelo generalice bien con datos no vistos y previene el sobreajuste.

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### Visualización de Datos
Utilizamos seaborn y matplotlib para generar visualizaciones que nos ayuden a explorar los datos e interpretar los resultados de nuestros modelos de machine learning.

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
import mlflow

# set the experiment id
mlflow.set_experiment(experiment_name="/Users/<username>/mlflow")

mlflow.autolog()

# Carga de Datos y Análisis Inicial con Pandas

En esta sección, se carga el conjunto de datos desde una URL utilizando **Pandas** y se realiza un análisis inicial de los datos con la librería **ydata_profiling** para comprender mejor las características y la estructura del conjunto de datos.

## Lectura del Conjunto de Datos desde una URL
Utilizamos la función `read_csv` de **Pandas** para leer el archivo CSV que contiene los datos del conjunto de vinos. El archivo se encuentra alojado en el repositorio de la **UCI Machine Learning Repository**.

### URL de los Datos:
- URL: [https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data)

El conjunto de datos incluye información química y visual sobre diferentes muestras de vino, clasificadas en tres tipos (clases). Para identificar claramente las columnas, las etiquetamos según la descripción proporcionada en el archivo de nombres asociado.

### Etiquetas de las columnas:
- **Class**: Clase de vino (1, 2 o 3)
- **Alcohol**: Contenido de alcohol
- **Malic_acid**: Ácido málico
- **Ash**: Ceniza
- **Alcalinity_of_ash**: Alcalinidad de la ceniza
- **Magnesium**: Magnesio
- **Total_phenols**: Fenoles totales
- **Flavanoids**: Flavonoides
- **Nonflavanoid_phenols**: Fenoles no flavonoides
- **Proanthocyanins**: Proantocianidinas
- **Color_intensity**: Intensidad del color
- **Hue**: Tono
- **OD280_OD315_of_diluted_wines**: OD280/OD315 de vinos diluidos (proporción entre las absorbancias a 280 y 315 nm)
- **Proline**: Prolina (aminoácido)


In [0]:
# Leer el archivo CSV desde la URL usando pandas
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
columns = ['Class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium', 
           'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins', 
           'Color_intensity', 'Hue', 'OD280_OD315_of_diluted_wines', 'Proline']

df_pandas = pd.read_csv(url, names=columns)



In [0]:
pfr = ProfileReport(df_pandas)
pfr

### Identificación de valores atípicos utilizando el método IQR

En este apartado, se implementa el **método del Rango Intercuartílico (IQR)** para identificar posibles valores atípicos en nuestro conjunto de datos.

El método IQR es una técnica ampliamente utilizada para detectar valores que se encuentran significativamente alejados del rango intercuartílico, definido como la diferencia entre el tercer cuartil (Q3) y el primer cuartil (Q1) de cada columna. Los valores que se encuentran por debajo de **Q1 - 1.5 * IQR** o por encima de **Q3 + 1.5 * IQR** son considerados valores atípicos.

El proceso consta de los siguientes pasos:

1. **Cálculo de los cuartiles Q1 y Q3:**
   Se calculan los cuartiles primero y tercero de cada columna numérica del DataFrame para definir el rango intercuartílico.

2. **Cálculo del IQR:**
   El **IQR** se obtiene como la diferencia entre los cuartiles Q3 y Q1.

3. **Detección de valores atípicos:**
   Se identifican los valores que están por debajo de **Q1 - 1.5 * IQR** o por encima de **Q3 + 1.5 * IQR** y se cuentan cuántos valores atípicos existen en cada columna.



In [0]:
# Identificación de valores atípicos utilizando el método IQR
Q1 = df_pandas.quantile(0.25)
Q3 = df_pandas.quantile(0.75)
IQR = Q3 - Q1
outliers = ((df_pandas < (Q1 - 1.5 * IQR)) | (df_pandas > (Q3 + 1.5 * IQR))).sum()
print("Valores atípicos por columna:\n", outliers)



### Imputación de valores atípicos utilizando la mediana

Después de identificar los valores atípicos, es crucial tratarlos adecuadamente para mejorar el rendimiento de los modelos predictivos. En este caso, hemos decidido **imputar** los valores atípicos con la **mediana** de cada columna.

La elección de la mediana es adecuada para este tipo de imputación, ya que, a diferencia de la media, no se ve influenciada por los valores extremos, lo que permite que los datos se mantengan más representativos y equilibrados.

#### Pasos del proceso:

1. **Cálculo de la mediana**:
   Para cada columna del conjunto de datos, se calcula la mediana de los valores. La mediana es el valor que divide a los datos en dos partes iguales y es resistente a los valores extremos.

2. **Imputación de valores atípicos**:
   Se reemplazan los valores que se encuentran fuera del rango definido por el método IQR con la mediana de la columna correspondiente.

3. **Actualización del DataFrame**:
   El DataFrame es actualizado con los nuevos valores imputados en lugar de los valores atípicos.



In [0]:
# Imputar los valores atípicos con la mediana
for column in df_pandas.columns:
    median = df_pandas[column].median()
    df_pandas[column] = df_pandas[column].mask((df_pandas[column] < (Q1[column] - 1.5 * IQR[column])) | (df_pandas[column] > (Q3[column] + 1.5 * IQR[column])), median)

print("Datos después de imputar valores atípicos:\n", df_pandas.describe())


### Creación de la sesión de Spark

Una de las primeras tareas en cualquier flujo de trabajo que utiliza **Apache Spark** es la creación de una sesión de Spark. Esta sesión es el punto de entrada principal para interactuar con los clústeres de Spark y realizar tareas de procesamiento de datos a gran escala.

En este proyecto, se utiliza **Spark** para procesar, transformar y analizar el conjunto de datos de clasificación de vinos, lo que permite aprovechar su capacidad para el manejo eficiente de grandes volúmenes de datos.

#### Descripción del proceso:

1. **Inicialización de SparkSession**:
   - La sesión de Spark se inicializa mediante el constructor `SparkSession.builder()`. 
   - Se define un nombre de aplicación, en este caso, `"WineClassification"`, para identificar esta sesión en el clúster de Spark.
   - La función `getOrCreate()` asegura que se cree una nueva sesión de Spark si no existe una, o reutiliza una existente si ya está en ejecución.

2. **AppName**:
   - Es una buena práctica dar un nombre a tu aplicación, lo cual facilita el seguimiento de las sesiones en un clúster, especialmente cuando hay múltiples aplicaciones ejecutándose de manera concurrente.


In [0]:
# Crear la sesión de Spark

spark = SparkSession.builder.appName("WineClassification").getOrCreate()

### Conversión de DataFrame de pandas a Spark DataFrame

En este paso, convertimos el **DataFrame de pandas** a un **Spark DataFrame**. Dado que Spark trabaja con grandes volúmenes de datos y distribuye las operaciones en múltiples nodos, es necesario transformar los datos desde pandas (que funciona en un solo nodo) a un formato distribuido compatible con Spark.

#### Descripción del proceso:

1. **Conversión de DataFrame de pandas a Spark DataFrame**:
   - El método `createDataFrame()` de la sesión de Spark se utiliza para convertir el DataFrame que ya hemos cargado y procesado en pandas.
   - Esta conversión es necesaria para que Spark pueda distribuir los datos y realizar operaciones a gran escala, como la clasificación, regresión y otras tareas de machine learning.

2. **Mostrar los primeros registros**:
   - Después de la conversión, utilizamos el método `show()` para visualizar las primeras filas del DataFrame en Spark.
   - En este caso, se muestran las primeras 5 filas para verificar que la conversión se realizó correctamente.



In [0]:
# Convertir DataFrame de pandas a Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)
df_spark.show(5)


### Creación del VectorAssembler para transformación de características

En esta sección, utilizamos **VectorAssembler**, una herramienta fundamental en **PySpark MLlib** para transformar múltiples columnas de características en una sola columna llamada `features`. Esta transformación es necesaria porque muchos algoritmos de machine learning en Spark requieren que los datos de entrada estén en forma de un solo vector.

#### Descripción del proceso:

1. **Selección de las columnas de características**:
   - Las características que se van a utilizar en el modelo se encuentran en todas las columnas excepto la columna `Class`, ya que esta última es nuestra variable objetivo.
   - Utilizamos `df_spark.columns[1:]` para seleccionar todas las columnas desde la segunda en adelante, excluyendo así la primera columna (`Class`), que corresponde a las etiquetas o clases.

2. **VectorAssembler**:
   - Este transformador toma una lista de columnas de entrada y las combina en una sola columna vectorial. En nuestro caso, queremos que todas las características queden condensadas en una nueva columna llamada `features`, que será la entrada para los algoritmos de clasificación.
   - Este paso es crucial porque Spark MLlib espera que las características de entrada estén en formato vectorial (denso o disperso) para la mayoría de los algoritmos de machine learning.

In [0]:
feature_columns = df_spark.columns[1:]  # Excluir la columna 'Class'
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

### División de los datos en conjuntos de entrenamiento y prueba

Una de las prácticas esenciales en la construcción de modelos de machine learning es dividir el conjunto de datos en dos partes: un conjunto de entrenamiento y un conjunto de prueba. Esta separación es importante para evaluar la capacidad predictiva del modelo en datos no vistos, lo que evita el sobreajuste y proporciona una mejor estimación del rendimiento en el mundo real.

En este paso, utilizamos el método **randomSplit** de PySpark para realizar esta división.

#### Descripción del proceso:

1. **randomSplit**:
   - Este método permite dividir un **DataFrame** en varios subconjuntos de forma aleatoria, basándose en los pesos que le proporcionemos.
   - En nuestro caso, queremos dividir los datos en un 80% para entrenamiento y un 20% para prueba.
   - Además, utilizamos una semilla fija (**seed**) para garantizar que la división sea reproducible. Esto es útil cuando quieres obtener los mismos subconjuntos al ejecutar el código varias veces.

2. **Conjunto de entrenamiento**:
   - El conjunto de entrenamiento se utiliza para ajustar (entrenar) el modelo, es decir, para enseñarle a reconocer patrones en los datos.

3. **Conjunto de prueba**:
   - El conjunto de prueba se reserva para evaluar el rendimiento del modelo después del entrenamiento. Este conjunto no se utiliza durante el ajuste del modelo, lo que permite evaluar de manera imparcial el desempeño del modelo en datos no vistos.


In [0]:
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=1234)

### Creación y entrenamiento de varios modelos de clasificación utilizando pipelines

En este paso, estamos construyendo tres modelos de clasificación diferentes para abordar el problema de clasificación multiclase en el dataset de vinos. Utilizamos **pipelines** de PySpark para organizar el flujo de trabajo, asegurando que el proceso de transformación y entrenamiento sea eficiente y reproducible.

#### Modelos utilizados:

1. **Regresión Logística (Logistic Regression)**:
   - Este modelo es uno de los métodos más comunes para la clasificación. Aunque suele utilizarse para problemas binarios, también puede adaptarse para problemas de clasificación multiclase utilizando una estrategia de softmax.
   - **Ventajas**: Simplicidad y eficiencia para conjuntos de datos linealmente separables.

2. **Árbol de Decisión (Decision Tree)**:
   - Los árboles de decisión son modelos interpretables que dividen los datos en subgrupos basados en características, realizando decisiones simples en cada paso.
   - **Ventajas**: Su facilidad para manejar relaciones no lineales y su capacidad de proporcionar interpretabilidad en los resultados.

3. **Bosque Aleatorio (Random Forest)**:
   - Este modelo es un conjunto de múltiples árboles de decisión entrenados en diferentes subconjuntos de los datos. Combina las predicciones de múltiples árboles para mejorar la precisión y reducir el sobreajuste.
   - **Ventajas**: Generalmente proporciona buenos resultados en una amplia variedad de tareas de clasificación, y es menos propenso al sobreajuste.

#### Implementación de los pipelines:

En PySpark, un **pipeline** es una secuencia de etapas, donde cada etapa es un transformador o un estimador. Esto permite encapsular tanto el preprocesamiento como el entrenamiento en una estructura clara y organizada.

#### Descripción del código:

1. **Diccionario de modelos**:
   - Creamos un diccionario llamado **models** que contiene los tres algoritmos que queremos entrenar. Cada modelo está configurado con la columna de etiquetas (`labelCol`) y la columna de características (`featuresCol`).
   - La ventaja de organizar los modelos en un diccionario es que nos permite iterar fácilmente sobre ellos para su entrenamiento y evaluación posterior.



In [0]:
# Crear y entrenar varios modelos de clasificación utilizando pipelines
models = {
    "Logistic Regression": LogisticRegression(labelCol="Class", featuresCol="features"),
    "Decision Tree": DecisionTreeClassifier(labelCol="Class", featuresCol="features"),
    "Random Forest": RandomForestClassifier(labelCol="Class", featuresCol="features")
}

### Evaluadores de métricas adicionales

En esta sección, se configuran varios **evaluadores** para medir el rendimiento de los modelos de clasificación que hemos creado. Evaluar correctamente el modelo es crucial para comprender qué tan bien se ajusta a los datos y cómo generaliza en los datos de prueba. Aquí se utilizan métricas que son comunes en problemas de clasificación, tales como la precisión, el recall, el F1 score y el área bajo la curva ROC.

#### Métricas utilizadas:

1. **Precisión ponderada (Weighted Precision)**:
   - La precisión mide la proporción de verdaderos positivos sobre todas las instancias clasificadas como positivas. La versión "ponderada" toma en cuenta el desbalance en las clases, lo que es importante en problemas de clasificación multiclase.
   - **¿Por qué es importante?**: Nos ayuda a entender cuántas predicciones hechas por el modelo son realmente correctas, considerando el desbalance de clases.

2. **Recall ponderado (Weighted Recall)**:
   - El recall mide la proporción de verdaderos positivos sobre todas las instancias que son realmente positivas. Al igual que la precisión, la versión "ponderada" ajusta este valor teniendo en cuenta el desbalance en las clases.
   - **¿Por qué es importante?**: El recall es crucial cuando los falsos negativos son costosos. Un alto recall indica que el modelo captura correctamente la mayoría de las instancias positivas.

3. **F1 score**:
   - El F1 score es la media armónica entre la precisión y el recall. Proporciona un balance entre ambos, siendo útil cuando es importante encontrar un equilibrio entre precisión y recall.
   - **¿Por qué es importante?**: El F1 score es una métrica robusta que equilibra la precisión y el recall, ideal en escenarios donde ambos son igual de importantes.

4. **Área bajo la curva ROC (AUC-ROC)**:
   - Aunque generalmente se usa para problemas binarios, también se puede adaptar para problemas multiclase. El área bajo la curva ROC (AUC-ROC) mide la capacidad del modelo para distinguir entre las diferentes clases.
   - **¿Por qué es importante?**: El AUC-ROC es particularmente útil para evaluar la calidad de las predicciones de probabilidad que genera el modelo.



In [0]:
# Evaluadores para métricas adicionales
precision_evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="f1")
# AUC-ROC es más relevante para problemas binarios, pero se puede adaptar para problemas multiclase
auc_evaluator = BinaryClassificationEvaluator(labelCol="Class", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Diccionario para almacenar los resultados
results = {}

### Entrenamiento y Evaluación de Modelos con Pipelines

En esta sección, entrenamos y evaluamos varios modelos de clasificación utilizando **pipelines** en PySpark. El pipeline es una herramienta muy poderosa, ya que nos permite definir un flujo de trabajo que combina el ensamblaje de las características (`VectorAssembler`) y el modelo de clasificación seleccionado en una sola estructura. Esto facilita la replicación del flujo para diferentes modelos de manera eficiente.

#### Modelos Utilizados:
Se han considerado tres modelos de clasificación para este problema de clasificación multiclase:
1. **Regresión Logística (Logistic Regression)**
2. **Árbol de Decisión (Decision Tree)**
3. **Bosque Aleatorio (Random Forest)**

Estos modelos se seleccionaron por su rendimiento en problemas de clasificación y su capacidad para capturar diferentes relaciones en los datos:
- **Regresión Logística** es ideal para problemas donde las clases son linealmente separables.
- **Árbol de Decisión** permite capturar relaciones no lineales y es fácil de interpretar.
- **Bosque Aleatorio** mejora el rendimiento mediante la agregación de múltiples árboles de decisión, reduciendo la varianza.

#### Flujo de Trabajo del Pipeline:
1. **Definir el Pipeline**: Para cada modelo, definimos un pipeline que incluye dos etapas: 
   - Ensamblaje de características (`VectorAssembler`) que convierte las columnas de entrada en una única columna de características.
   - El modelo de clasificación que se está evaluando.

2. **Entrenar el Pipeline**: Se entrena el pipeline utilizando el conjunto de datos de entrenamiento. Esto entrena el ensamblador y el modelo en una sola llamada.

3. **Hacer Predicciones**: Una vez que el pipeline ha sido entrenado, se realizan predicciones sobre el conjunto de datos de prueba.

4. **Evaluar el Modelo**:
   - Se utilizan evaluadores de clasificación multiclase para calcular las siguientes métricas:
     - **Precisión (Accuracy)**: Proporción de instancias clasificadas correctamente.
     - **Precisión ponderada (Precision)**: Proporción de predicciones correctas dentro de todas las predicciones realizadas para una clase.
     - **Recall ponderado (Recall)**: Proporción de verdaderos positivos capturados por el modelo.
     - **F1 Score**: Promedio armónico entre precisión y recall.
     - **Área bajo la curva ROC (AUC)**: Utilizado en clasificación binaria, se muestra solo si el modelo predice dos clases.

5. **Resultados**: Los resultados se almacenan en un diccionario para cada modelo, con las métricas de rendimiento clave. Esto permite comparar fácilmente el rendimiento de los distintos modelos.



In [0]:
for model_name, model in models.items():
    # Definir el pipeline
    evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy")
    pipeline = Pipeline(stages=[assembler, model])
    
    # Entrenar el pipeline
    pipeline_model = pipeline.fit(train_data)
    
    # Hacer predicciones
    predictions = pipeline_model.transform(test_data)
    
    # Evaluar el modelo
    accuracy = evaluator.evaluate(predictions)
    precision = precision_evaluator.evaluate(predictions)
    recall = recall_evaluator.evaluate(predictions)
    f1_score = f1_evaluator.evaluate(predictions)
    
    # Use AUC evaluator only for binary classification
    if len(predictions.select("rawPrediction").first()[0]) == 2:
        auc = auc_evaluator.evaluate(predictions)
    else:
        auc = None
    
    results[model_name] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "auc": auc
    }
    
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}, AUC: {auc if auc is not None else 'N/A'}")


### Selección del mejor modelo basado en F1-Score

Después de entrenar y evaluar varios modelos de clasificación utilizando pipelines, el siguiente paso es identificar el modelo que ha tenido el mejor rendimiento según una métrica de evaluación específica. En este caso, utilizamos el **F1-Score** como criterio principal para seleccionar el mejor modelo.

#### ¿Por qué F1-Score?

El **F1-Score** es una métrica robusta que equilibra dos métricas importantes: la **precisión** (qué porcentaje de nuestras predicciones positivas son correctas) y el **recall** (qué porcentaje de los positivos reales fueron capturados por el modelo). Es especialmente útil cuando el balance entre estas dos métricas es crucial, como en problemas donde tanto los falsos positivos como los falsos negativos son importantes.

El F1-Score se utiliza generalmente cuando tenemos un conjunto de datos desbalanceado o cuando tanto los falsos positivos como los falsos negativos deben minimizarse. Por eso, es una excelente métrica para seleccionar el modelo que mejor generaliza a datos no vistos.

#### Descripción del proceso:

1. **Buscar el mejor modelo**:
   - Usamos la función `max()` para iterar a través del diccionario `results` y seleccionar el modelo con el valor más alto de **F1-Score**.
   - El **lambda** es una función anónima que nos permite acceder al valor del F1-Score para cada modelo en el diccionario.

2. **Imprimir el mejor modelo**:
   - Una vez identificado el mejor modelo, se imprime el nombre del modelo junto con su F1-Score, lo que permite identificar cuál de los algoritmos de clasificación fue el más efectivo en este experimento.


In [0]:
# Seleccionar el mejor modelo basado en F1-Score (puedes cambiar esto según tu criterio)
best_model_name = max(results, key=lambda x: results[x]['f1_score'])
print(f"Mejor modelo: {best_model_name} con un F1-Score de {results[best_model_name]['f1_score']:.4f}")

### Visualización y documentación de las métricas del modelo

Después de entrenar y evaluar varios modelos de clasificación, es crucial visualizar y comparar las métricas clave para identificar el modelo que ofrece el mejor rendimiento. En esta sección, se crearán gráficas para representar visualmente los resultados de las métricas, y se documentarán en un archivo de texto para futuras referencias.

#### Visualización de métricas

Utilizamos **matplotlib** para crear gráficos de barras que muestran las siguientes métricas para cada modelo:
1. **Precisión** (Accuracy)
2. **Precisión ponderada** (Precision)
3. **Exhaustividad** (Recall)
4. **F1-Score**
5. **AUC-ROC**

Cada una de estas métricas es esencial para entender el rendimiento del modelo en tareas de clasificación. Además, los gráficos nos ayudan a comparar fácilmente el rendimiento entre los diferentes modelos.

#### Explicación de las métricas visualizadas:
- **Precisión (Accuracy)**: Proporción de instancias correctamente clasificadas.
- **Precisión ponderada (Precision)**: Proporción de predicciones correctas sobre las instancias predichas como positivas, ajustada por el desbalance de clases.
- **Recall ponderado (Recall)**: Proporción de instancias positivas correctamente identificadas por el modelo, también ajustada por el desbalance de clases.
- **F1-Score**: Promedio armónico entre precisión y recall, útil para evaluar el equilibrio entre ambas métricas.
- **AUC-ROC**: Área bajo la curva ROC, mide la capacidad del modelo para distinguir entre las clases en términos de probabilidad.



In [0]:
import matplotlib.pyplot as plt

# Extraer las métricas de los resultados y manejar valores None
model_names = list(results.keys())
accuracies = [results[model]['accuracy'] if results[model]['accuracy'] is not None else 0 for model in model_names]
precisions = [results[model]['precision'] if results[model]['precision'] is not None else 0 for model in model_names]
recalls = [results[model]['recall'] if results[model]['recall'] is not None else 0 for model in model_names]
f1_scores = [results[model]['f1_score'] if results[model]['f1_score'] is not None else 0 for model in model_names]
aucs = [results[model]['auc'] if results[model]['auc'] is not None else 0 for model in model_names]

# Crear una figura con subplots para cada métrica
fig, axs = plt.subplots(3, 2, figsize=(15, 18))

# Graficar la precisión
axs[0, 0].bar(model_names, accuracies, color=['blue', 'green', 'red'])
axs[0, 0].set_title('Precisión de Modelos de Clasificación')
axs[0, 0].set_xlabel('Modelos')
axs[0, 0].set_ylabel('Precisión')
axs[0, 0].set_ylim(0, 1)

# Graficar la precisión (Precision)
axs[0, 1].bar(model_names, precisions, color=['blue', 'green', 'red'])
axs[0, 1].set_title('Precisión (Precision) de Modelos de Clasificación')
axs[0, 1].set_xlabel('Modelos')
axs[0, 1].set_ylabel('Precisión')
axs[0, 1].set_ylim(0, 1)

# Graficar la exhaustividad (Recall)
axs[1, 0].bar(model_names, recalls, color=['blue', 'green', 'red'])
axs[1, 0].set_title('Exhaustividad (Recall) de Modelos de Clasificación')
axs[1, 0].set_xlabel('Modelos')
axs[1, 0].set_ylabel('Exhaustividad')
axs[1, 0].set_ylim(0, 1)

# Graficar el F1-Score
axs[1, 1].bar(model_names, f1_scores, color=['blue', 'green', 'red'])
axs[1, 1].set_title('F1-Score de Modelos de Clasificación')
axs[1, 1].set_xlabel('Modelos')
axs[1, 1].set_ylabel('F1-Score')
axs[1, 1].set_ylim(0, 1)

# Graficar el AUC-ROC
axs[2, 0].bar(model_names, aucs, color=['blue', 'green', 'red'])
axs[2, 0].set_title('AUC-ROC de Modelos de Clasificación')
axs[2, 0].set_xlabel('Modelos')
axs[2, 0].set_ylabel('AUC-ROC')
axs[2, 0].set_ylim(0, 1)

# Ajustar el layout
plt.tight_layout()

# Guardar la gráfica en un archivo
plt.savefig('model_comparison_metrics.png')

# Mostrar la gráfica (esto no funcionará en entornos no interactivos)
plt.show()

# Documentar los resultados y las evidencias
with open('model_results.txt', 'w') as f:
    for model_name, metrics in results.items():
        accuracy = metrics['accuracy'] if metrics['accuracy'] is not None else 0
        precision = metrics['precision'] if metrics['precision'] is not None else 0
        recall = metrics['recall'] if metrics['recall'] is not None else 0
        f1_score = metrics['f1_score'] if metrics['f1_score'] is not None else 0
        auc = metrics['auc'] if metrics['auc'] is not None else 0
        f.write(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}, AUC: {auc:.4f}\n")
    f.write(f"\nMejor modelo: {best_model_name} con un F1-Score de {results[best_model_name]['f1_score']:.4f}\n")

### Entrenamiento del modelo Random Forest y predicción sobre nuevas muestras

En esta sección, entrenamos un modelo de **Random Forest** para clasificar las muestras de vino basándonos en sus características químicas. El modelo se entrena utilizando **pipelines** para un flujo más eficiente y reproducible. Además, probamos el modelo con nuevas muestras de vino y mostramos las predicciones junto con las probabilidades asociadas a cada clase.

#### 1. Selección de características y división de los datos
Primero, seleccionamos las columnas correspondientes a las características (todas excepto la columna 'Class', que es la variable objetivo). Luego, dividimos los datos en conjuntos de entrenamiento (80%) y prueba (20%) de forma aleatoria para evaluar el modelo.



In [0]:
# Seleccionar las características y la etiqueta
feature_columns = df_spark.columns[1:]  # Excluir la columna 'Class'
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=1234)

### 2. Entrenamiento del modelo Random Forest utilizando pipelines
Utilizamos el algoritmo Random Forest, que es adecuado para tareas de clasificación con datos complejos debido a su capacidad para combinar múltiples árboles de decisión y mejorar la generalización. El pipeline combina la transformación de las características (VectorAssembler) y el modelo de clasificación en un solo flujo de trabajo.

In [0]:
# Crear y entrenar el modelo de Random Forest utilizando pipelines
rf = RandomForestClassifier(labelCol="Class", featuresCol="features", probabilityCol="probability")
pipeline = Pipeline(stages=[assembler, rf])
pipeline_model = pipeline.fit(train_data)

### 3. Evaluación del modelo
Una vez entrenado el modelo, lo evaluamos en el conjunto de datos de prueba utilizando la métrica de precisión (accuracy) para medir qué tan bien el modelo ha clasificado las muestras.

In [0]:
# Evaluar el modelo en el conjunto de prueba
predictions_test = pipeline_model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_test)
print(f"Random Forest Accuracy: {accuracy:.4f}")

### 4. Predicción sobre nuevas muestras
A continuación, realizamos predicciones sobre dos nuevas muestras de vino para determinar su clase. Las nuevas muestras son ingresadas manualmente en forma de lista y convertidas a un DataFrame de Spark. Luego, utilizamos el pipeline entrenado para hacer las predicciones.

In [0]:
# Nuevas muestras de vino
new_samples = [
    [13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285],
    [12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520]
]

# Crear un DataFrame de Spark para las nuevas muestras
new_samples_df = spark.createDataFrame(new_samples, schema=feature_columns)

# Hacer predicciones sobre las nuevas muestras directamente con el pipeline
predictions_new_samples = pipeline_model.transform(new_samples_df)

# Mostrar los resultados de las predicciones con las probabilidades
predictions_new_samples.select("features", "prediction", "probability").show(truncate=False)

### Aplicación de PCA para reducir la dimensionalidad

En esta sección, se implementa el uso de **Análisis de Componentes Principales (PCA)** para reducir la dimensionalidad de los datos. Esta técnica es útil cuando las características originales tienen alta correlación, lo que puede generar redundancia y afectar el rendimiento del modelo. PCA transforma las características originales en un conjunto de nuevas características (componentes principales) que capturan la mayor cantidad de variación posible en los datos con un número reducido de dimensiones.



In [0]:
# Convertir DataFrame de pandas a Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Seleccionar las características y la etiqueta
feature_columns = df_spark.columns[1:]  # Excluir la columna 'Class'
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Aplicar PCA para reducir la dimensionalidad
pca = PCA(k=5, inputCol="features", outputCol="pcaFeatures")

# Crear y entrenar el modelo de Random Forest utilizando pipelines
rf = RandomForestClassifier(labelCol="Class", featuresCol="pcaFeatures")
pipeline = Pipeline(stages=[assembler, pca, rf])

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=1234)

# Entrenar el pipeline
pipeline_model = pipeline.fit(train_data)

# Hacer predicciones sobre el conjunto de prueba
predictions = pipeline_model.transform(test_data)

# Evaluar el modelo
evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Random Forest with PCA Accuracy: {accuracy:.4f}")

# Nuevas muestras de vino
new_samples = [
    [13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285],
    [12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520]
]

# Crear un DataFrame de Spark para las nuevas muestras
new_samples_df = spark.createDataFrame(new_samples, schema=feature_columns)

# Hacer predicciones sobre las nuevas muestras directamente con el pipeline
predictions_new_samples = pipeline_model.transform(new_samples_df)

# Mostrar los resultados de las predicciones con las probabilidades
predictions_new_samples.select("pcaFeatures", "prediction", "probability").show(truncate=False)


### Aplicación de PCA y Cross-Validation con ajuste de hiperparámetros

En esta sección, no solo aplicamos **Análisis de Componentes Principales (PCA)** para reducir la dimensionalidad, sino que también utilizamos un proceso de **validación cruzada (Cross-Validation)** para ajustar los hiperparámetros del modelo de **Random Forest**. Este enfoque nos permite encontrar la mejor combinación de parámetros para maximizar la precisión del modelo.


In [0]:

# Convertir DataFrame de pandas a Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Seleccionar las características y la etiqueta
feature_columns = df_spark.columns[1:]  # Excluir la columna 'Class'
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Aplicar PCA para reducir la dimensionalidad
pca = PCA(k=5, inputCol="features", outputCol="pcaFeatures")

# Crear y entrenar el modelo de Random Forest utilizando pipelines
rf = RandomForestClassifier(labelCol="Class", featuresCol="pcaFeatures")
pipeline = Pipeline(stages=[assembler, pca, rf])

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=1234)

# Crear una cuadrícula de hiperparámetros para Random Forest
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 20, 30])
             .addGrid(rf.maxDepth, [5, 10, 15])
             .build())

# Crear el CrossValidator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy"),
                          numFolds=5)

# Entrenar el modelo utilizando CrossValidator
cv_model = crossval.fit(train_data)

# Hacer predicciones sobre el conjunto de prueba
predictions = cv_model.transform(test_data)

# Evaluar el modelo
evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Random Forest with PCA and Cross-Validation Accuracy: {accuracy:.4f}")

# Nuevas muestras de vino
new_samples = [
    [13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285],
    [12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520]
]

# Crear un DataFrame de Spark para las nuevas muestras
new_samples_df = spark.createDataFrame(new_samples, schema=feature_columns)

# Hacer predicciones sobre las nuevas muestras directamente con el pipeline
predictions_new_samples = cv_model.transform(new_samples_df)

# Mostrar los resultados de las predicciones con las probabilidades
predictions_new_samples.select("pcaFeatures", "prediction", "probability").show(truncate=False)



In [0]:
# Detener la sesión de Spark
spark.stop()